# [Inference] Gemma 7b Bangla News Summarization

In [1]:
!nvidia-smi

Mon Mar 11 18:40:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:01:00.0 Off |                  N/A |
|  0%   49C    P8              28W / 300W |      1MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
!pip install -qqq kaggle pandas evaluate rouge_score huggingface_hub torch==2.0.1 loralib==0.1.1 einops==0.6.1 pandas
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy

In [7]:
!sudo apt-get install unzip > /dev/null

In [4]:
!mkdir ~/.kaggle
!cp /home/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d prithwirajsust/bengali-news-summarization-dataset
!unzip /home/bengali-news-summarization-dataset.zip

  0%|                                               | 0.00/2.12M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 2.12M/2.12M [00:00<00:00, 27.7MB/s]
Archive:  /home/bengali-news-summarization-dataset.zip
  inflating: Bengali-News-Summarization-Dataset/article.txt  
  inflating: Bengali-News-Summarization-Dataset/summary.txt  


In [10]:
import numpy as np
import random
import pandas as pd
import os

def split_dataset(path):
    dataset = open(path, encoding='utf-8').read().split('\n')

    dataset_len = len(dataset)
    print("There are %s summary-article pairs" % dataset_len)

    # Random shuffle data
    random.seed(11)
    dataset = np.array(random.sample(dataset, len(dataset)))

    train_size = int(dataset_len * 0.7)
    eval_size = int(dataset_len * 0.2)
    train, val, test = dataset[:train_size], dataset[train_size:train_size + eval_size], dataset[train_size + eval_size:]
    return train, val, test

def write_csv(filename, enc_data, dec_data):
    df = pd.DataFrame({'Text': enc_data, 'Summary': dec_data})
    df.to_csv(filename, index=False)

def main():
    summary_datapath = '/home/Bengali-News-Summarization-Dataset/summary.txt'
    article_datapath = '/home/Bengali-News-Summarization-Dataset/article.txt'
    
    dec_train, dec_val, dec_test = split_dataset(summary_datapath)
    enc_train, enc_val, enc_test = split_dataset(article_datapath)
    
    # Create the directory if it doesn't exist
    directory = './datasetFull'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    write_csv(os.path.join(directory, 'train.csv'), enc_train, dec_train)
    write_csv(os.path.join(directory, 'val.csv'), enc_val, dec_val)
    write_csv(os.path.join(directory, 'test.csv'), enc_test, dec_test)
    
    print("Finished splitting and saving dataset!")

if __name__ == "__main__":
    main()

There are 19097 summary-article pairs
There are 19097 summary-article pairs
Finished splitting and saving dataset!


In [12]:
from huggingface_hub import login
login()

# Inference Code

In [13]:
import pandas as pd
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [14]:
model = "samanjoy2/gemma7b-it_banglaNewsSum"
MODEL_NAME = model
PEFT_MODEL = MODEL_NAME

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

In [44]:
generation_config = model.generation_config
generation_config.max_new_tokens = 50
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [45]:
train_df = pd.read_csv("/home/datasetFull/train.csv", encoding = 'utf-8')
train_df.columns = [str(q) for q in train_df.columns]

val_df = pd.read_csv("/home/datasetFull/val.csv", encoding = 'utf-8')
val_df.columns = [str(q) for q in val_df.columns]

test_df = pd.read_csv("/home/datasetFull/test.csv", encoding = 'utf-8')
test_df.columns = [str(q) for q in test_df.columns]

train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)

In [46]:
import pandas as pd
import torch
from tqdm import tqdm

results_list = []  

for i in tqdm(range(len(test_df)), desc='Processing'):
    prompt = f"""
    <start_of_turn>
    Provide a concise Bengali summary of the following news article, focusing on the most important information. 

    Note:
    Use only Bengali for the summary.
    Stay objective and factual in your summary.

    ####

    Article: {test_df["Text"].values[i]}

    ####
    <end_of_turn>
    """

    device = "cuda" if torch.cuda.is_available() else "cpu"  # Check if GPU is available
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )

    try:
        predicted_summary = tokenizer.decode(outputs[0], skip_special_tokens=True).split('Summary:')[1].split('\n')[0].strip()
    except Exception as e:
        predicted_summary = ''
    
    true_summary = test_df["Summary"].values[i]
    
    results_list.append({'Prediction': predicted_summary, 'True_Label': true_summary})
    
    print(f"Predicted Summary: {predicted_summary}")
    print(f"True Label: {true_summary}")
    print()

results_df = pd.DataFrame(results_list)

results_df.to_csv('predictions_and_true_labels.csv', index=False)

Processing:   0%|          | 0/1911 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Processing:   0%|          | 1/1911 [00:02<1:05:54,  2.07s/it]

Predicted Summary: নাশকতা ও রাষ্ট্রদ্রোহের দশ মামলায় আদালতে হাজির
True Label: ১০ মামলায় খালেদার জামিন



Processing:   0%|          | 2/1911 [00:04<1:06:00,  2.07s/it]

Predicted Summary: ভুল চুলের ছাঁট
True Label: কেশশৈলীর ৫ ভুল: দেখতে লাগবে বুড়োটে



Processing:   0%|          | 3/1911 [00:06<1:05:56,  2.07s/it]

...

Predicted Summary: গত বছরের শুরুতে ওয়ানডেতে অভিষিক্ত হওয়া আরাফাত সানি
True Label: বিশ্বকাপ দলে ডাক পেয়ে রোমাঞ্চিত সানি



Processing: 100%|█████████▉| 1910/1911 [1:05:55<00:02,  2.05s/it]

Predicted Summary: 
True Label: যুক্তরাষ্ট্রে সেক্টর কমান্ডার্স ফোরামের মতবিনিময়



Processing: 100%|██████████| 1911/1911 [1:05:57<00:00,  2.07s/it]

Predicted Summary: জাহাঙ্গীরনগর বিশ্ববিদ্যালয় থেকে এক শিক্ষার্থীকে আইনশৃঙ্খলা বাহিনী পরিচয়ে তুলে নিয়ে য
True Label: জাবি শিক্ষার্থীকে নিয়ে গেছে আইনশৃঙ্খলা বাহিনী



In [59]:
import csv
import evaluate
from tqdm import tqdm

bleu = evaluate.load("bleu")

predictions = []
true_labels = []
with open("predictions_and_true_labels.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader:
        predictions.append(row[0])
        true_labels.append(row[1])

bleu_scores = []
with tqdm(total=len(predictions), desc="Calculating BLEU scores") as pbar:
    for pred, true in zip(predictions, true_labels):
        if not pred:
            bleu_scores.append({'bleu': 0, 'precisions': [0, 0, 0, 0], 'brevity_penalty': 0, 'length_ratio': 0, 'translation_length': 0, 'reference_length': 0})
            pbar.update(1)
            continue
        references = [[true]]
        bleu_result = bleu.compute(predictions=[pred], references=references)
        bleu_scores.append(bleu_result)
        pbar.update(1)

valid_bleu_scores = [score['bleu'] for score in bleu_scores if score['bleu'] > 0]  # Exclude empty predictions from average calculation
average_bleu = sum(valid_bleu_scores) / len(valid_bleu_scores)

print("Individual BLEU Scores:")
for i, score in enumerate(bleu_scores, start=1):
    print(f"Sentence {i}: {score}")
print("\nAverage BLEU Score:", average_bleu)

Calculating BLEU scores: 100%|██████████| 1911/1911 [28:24<00:00,  1.12it/s]

Individual BLEU Scores:
Sentence 1: {'bleu': 0.0, 'precisions': [0.14285714285714285, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.75, 'translation_length': 7, 'reference_length': 4}
Sentence 2: {'bleu': 0.0, 'precisions': [0.3333333333333333, 0.0, 0.0, 0.0], 'brevity_penalty': 0.2635971381157267, 'length_ratio': 0.42857142857142855, 'translation_length': 3, 'reference_length': 7}
Sentence 3: {'bleu': 0.0, 'precisions': [0.16666666666666666, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.2, 'translation_length': 6, 'reference_length': 5}
Sentence 4: {'bleu': 0.0, 'precisions': [0.0, 0.0, 0.0, 0.0], 'brevity_penalty': 0.22313016014842982, 'length_ratio': 0.4, 'translation_length': 2, 'reference_length': 5}
Sentence 5: {'bleu': 0.0, 'precisions': [0.0, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.2, 'translation_length': 6, 'reference_length': 5}
Sentence 6: {'bleu': 0.0, 'precisions': [0.0, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.25

In [ ]:
# Average BLEU Score: 0.40414279792780694